In [16]:
import os
import pandas as pd

pd.set_option('display.max_columns', None)

# 1. 设置路径
data_path = r'E:\shenzhen_house_price_project\data\raw'
os.chdir(data_path)

# 2. 导入所有 Excel 文件
files = [f for f in os.listdir() if f.endswith('.xlsx')]
dfs = []

for f in files:
    print("正在读取：", f)
    df = pd.read_excel(f)
    df['source_file'] = f
    dfs.append(df)


# 3. 合并
df_all = pd.concat(dfs, ignore_index=True)
print("合并后数据量：", df_all.shape)

# 4. 保存合并结果
df_all.to_excel(r'E:\shenzhen_house_price_project\data\clean\合并数据.xlsx', index=False)
print("已保存：合并数据.xlsx")



正在读取： sz_baoan_2025.xlsx
正在读取： sz_dapengxinqu_2025.xlsx
正在读取： sz_futian_2025.xlsx
正在读取： sz_guangming_2025.xlsx
正在读取： sz_longgang_2025.xlsx
正在读取： sz_longhua_2025.xlsx
正在读取： sz_luohu_2025.xlsx
正在读取： sz_nanshan_2025.xlsx
正在读取： sz_pingshan_2025.xlsx
正在读取： sz_yantian_2025.xlsx
合并后数据量： (46945, 13)
已保存：合并数据.xlsx


In [22]:
import pandas as pd
import sqlite3
# === 0. 读取数据 ===
file_path = r'E:\shenzhen_house_price_project\data\clean\合并数据.xlsx'
df = pd.read_excel(file_path)
print("原始数据规模：", df.shape)
print("="*100)

# === 1. 去重 ===
print("重复行数量：", df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)
print("去重后数据规模：", df.shape)
print("="*100)
# === 2. 删除无用字段 ===
drop_cols = ['title', 'address', 'label']
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')


原始数据规模： (46945, 13)
重复行数量： 15064
去重后数据规模： (31881, 13)


In [25]:
# === 3. 修改列名（去除单位、空格替换下划线） ===
df.columns = df.columns.str.replace(r'（.*?）|\(.*?\)', '', regex=True)
df.columns = [c.strip().replace(' ', '_') for c in df.columns]
print("列名清洗完成：", df.columns.tolist())
print("="*100)

列名清洗完成： ['district', 'roomnum', 'hall', 'AREA', 'C_floor', 'school', 'subway', 'per_price', 'total_price', 'source_file']


In [26]:
# === 4. 查看异常值情况 ===
print("卧室数分布：")
print(df['roomnum'].value_counts(dropna=False))
print('-'*60)

print("楼层缺失数量：", df['C_floor'].isna().sum())
print('-'*60)

# === 5. 删除异常值 ===
# 卧室数不能为 0
df = df[df['roomnum'] != 0]

# 楼层不能为空或空白
df = df[df['C_floor'].notna()]
df = df[df['C_floor'].astype(str).str.strip() != '']

df = df.reset_index(drop=True)
print("清洗完成后数据量：", len(df))
print("="*100)



卧室数分布：
roomnum
3.0    13809
4.0     6548
2.0     5819
1.0     2175
5.0     1932
NaN      910
6.0      426
7.0      145
8.0       77
9.0       36
0.0        4
Name: count, dtype: int64
------------------------------------------------------------
楼层缺失数量： 1973
------------------------------------------------------------
清洗完成后数据量： 29904


In [27]:
# === 6. 拼音 -> 中文转换 ===
dist_dict = {
    'longgang': '龙岗', 'longhua': '龙华', 'baoan': '宝安',
    'nanshan': '南山', 'futian': '福田', 'luohu': '罗湖',
    'pingshan': '坪山', 'guangming': '光明', 'yantian': '盐田', 'dapengxinqu': '大鹏新区'
}

df['district'] = df['district'].apply(lambda x: dist_dict.get(x, x))
print(df.head(2))
# === 7. 单位换算 ===
# 将 per_price 从 元/㎡ 转换为 万元/㎡
df['per_price'] = df['per_price'] / 10000

# 验证转换结果
print("✅ 单位转换后 per_price 前5行（应为 5~10 左右）：")
print(df['per_price'].head().round(2))


  district  roomnum  hall  AREA C_floor  school  subway  per_price  \
0       宝安      2.0   2.0  52.2  middle       0       1      20114   
1       宝安      2.0   2.0  60.0  middle       0       1      20000   

   total_price         source_file  
0        105.0  sz_baoan_2025.xlsx  
1        120.0  sz_baoan_2025.xlsx  
✅ 单位转换后 per_price 前5行（应为 5~10 左右）：
0    2.01
1    2.00
2    1.60
3    1.09
4    3.21
Name: per_price, dtype: float64


In [28]:
# === 8. 查看关键字段描述 ===
print(df[['roomnum', 'hall', 'AREA', 'per_price', 'total_price']].describe())
print("="*100)

# === 9. 查看分类变量取值 ===
con = sqlite3.connect(':memory:')
df.to_sql('df', con, index=False)

for col in ['district', 'roomnum', 'hall', 'C_floor', 'school', 'subway']:
    print(f"字段：{col}")
    print(pd.read_sql_query(f"SELECT DISTINCT {col} FROM df", con))
    print('-'*20)
print("="*100)


# === 11. 保存清洗后数据 ===
save_path = r'E:\shenzhen_house_price_project\data\clean'
df.to_excel(save_path + r'\清洗后数据.xlsx', index=False)
print("已保存：清洗后数据.xlsx")


            roomnum         hall          AREA     per_price   total_price
count  29904.000000  29904.00000  29904.000000  29904.000000  29904.000000
mean       3.073067      1.80307    103.081402      5.695858    682.099839
std        1.095166      0.47922     52.367763      3.555464    870.308192
min        1.000000      0.00000     13.870000      0.620300     37.800000
25%        2.000000      2.00000     76.000000      3.429800    268.000000
50%        3.000000      2.00000     89.000000      4.635450    407.000000
75%        4.000000      2.00000    117.882500      6.818100    720.000000
max        9.000000      5.00000    960.000000     59.146900  16899.000000
字段：district
  district
0       宝安
1     大鹏新区
2       福田
3       光明
4       龙岗
5       龙华
6       罗湖
7       南山
8       坪山
9       盐田
--------------------
字段：roomnum
   roomnum
0      2.0
1      3.0
2      5.0
3      9.0
4      4.0
5      7.0
6      6.0
7      1.0
8      8.0
--------------------
字段：hall
   hall
0   2.0
1   1